In [1]:
from copy import deepcopy
import time
import math

# Color codes for better visualization
ansi_red = "\u001b[31m"
ansi_green = "\u001b[32m"
ansi_reset = "\u001b[0m"

class Checkers:
    def __init__(self):
        self.matrix = [[], [], [], [], [], [], [], []]
        self.current_turn = True  # True for player (O), False for AI (X)
        self.player_pieces = 12
        self.ai_pieces = 12
        
        # Initialize empty board
        for row in self.matrix:
            for _ in range(8):
                row.append("---")
        
        # Position pieces
        self.position_pieces()

    def position_pieces(self):
        # Place AI pieces (X) at top (rows 0-2)
        for i in range(3):
            for j in range(8):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = f"X{i}{j}"  # AI pieces (X)
        
        # Place player pieces (O) at bottom (rows 5-7)
        for i in range(5, 8):
            for j in range(8):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = f"O{i}{j}"  # Player pieces (O)

    def print_board(self):
        print("\n   " + "   ".join(str(i) for i in range(8)))
        for i, row in enumerate(self.matrix):
            print(i, end="  ")
            for cell in row:
                if cell[0] == 'X':
                    print(ansi_red + cell[0] + ansi_reset, end="   ")
                elif cell[0] == 'O':
                    print(ansi_green + cell[0] + ansi_reset, end="   ")
                else:
                    print(cell[0], end="   ")
            print()
        print()

    def get_player_move(self):
        while True:
            try:
                move = input("Enter your move (from_row,from_col to_row,to_col): ").strip()
                if not move:
                    print("Game ended!")
                    exit()
                
                from_pos, to_pos = move.split()
                old_i, old_j = map(int, from_pos.split(','))
                new_i, new_j = map(int, to_pos.split(','))
                
                # Validate move
                if self.is_valid_move(old_i, old_j, new_i, new_j, is_player=True):
                    self.make_move(old_i, old_j, new_i, new_j)
                    return
                else:
                    print("Invalid move! Try again.")
            except (ValueError, IndexError):
                print("Invalid input format! Use: from_row,from_col to_row,to_col")

    def is_valid_move(self, old_i, old_j, new_i, new_j, is_player):
        # Check if coordinates are within bounds
        if not (0 <= old_i < 8 and 0 <= old_j < 8 and 0 <= new_i < 8 and 0 <= new_j < 8):
            return False
        
        # Check if moving from own piece
        piece = self.matrix[old_i][old_j][0]
        if is_player and piece != 'O':
            return False
        if not is_player and piece != 'X':
            return False
        
        # Check if destination is empty
        if self.matrix[new_i][new_j] != "---":
            return False
        
        # Check if move is diagonal
        if abs(old_i - new_i) != 1 or abs(old_j - new_j) != 1:
            return False
        
        # Check direction (player O moves up, AI X moves down)
        if is_player and new_i >= old_i:  # Player can only move up (decreasing row)
            return False
        if not is_player and new_i <= old_i:  # AI can only move down (increasing row)
            return False
            
        return True

    def make_move(self, old_i, old_j, new_i, new_j):
        # Move the piece
        piece = self.matrix[old_i][old_j]
        self.matrix[old_i][old_j] = "---"
        self.matrix[new_i][new_j] = piece
        
        # Switch turns
        self.current_turn = not self.current_turn
        
        # Update piece counts
        self.update_piece_counts()

    def update_piece_counts(self):
        self.player_pieces = sum(1 for row in self.matrix for cell in row if cell[0] == 'O')
        self.ai_pieces = sum(1 for row in self.matrix for cell in row if cell[0] == 'X')

    def ai_move(self):
        # Find all possible moves
        moves = []
        for i in range(8):
            for j in range(8):
                if self.matrix[i][j][0] == 'X':
                    # Check possible moves for this AI piece (can only move down)
                    for di, dj in [(1, -1), (1, 1)]:  # X moves down (increasing row)
                        new_i, new_j = i + di, j + dj
                        if self.is_valid_move(i, j, new_i, new_j, is_player=False):
                            moves.append((i, j, new_i, new_j))
        
        if not moves:
            print("AI has no valid moves left!")
            return False
        
        # Choose first valid move (simple AI)
        move = moves[0]
        print(f"AI moves from ({move[0]},{move[1]}) to ({move[2]},{move[3]})")
        self.make_move(*move)
        return True

    def check_game_over(self):
        if self.player_pieces == 0:
            self.print_board()
            print("You have no pieces left. AI wins!")
            return True
        if self.ai_pieces == 0:
            self.print_board()
            print("AI has no pieces left. You win!")
            return True
        
        # Check if current player has no valid moves
        if self.current_turn:  # Player's turn
            if not any(self.is_valid_move(i, j, i-1, j-1, True) or 
                       self.is_valid_move(i, j, i-1, j+1, True)
                       for i in range(8) for j in range(8) if self.matrix[i][j][0] == 'O'):
                self.print_board()
                print("You have no valid moves left. AI wins!")
                return True
        else:  # AI's turn
            if not any(self.is_valid_move(i, j, i+1, j-1, False) or 
                       self.is_valid_move(i, j, i+1, j+1, False)
                       for i in range(8) for j in range(8) if self.matrix[i][j][0] == 'X'):
                self.print_board()
                print("AI has no valid moves left. You win!")
                return True
                
        return False

    def play(self):
        print("Welcome to Simplified Checkers!")
        print("You are O (green), AI is X (red)")
        print("Enter moves like: '5,2 4,3' to move from (5,2) to (4,3)")
        print("Player pieces (O) move upward, AI pieces (X) move downward")
        
        while True:
            self.print_board()
            
            if self.check_game_over():
                break
                
            if self.current_turn:
                print("Your turn (O)")
                self.get_player_move()
            else:
                print("AI's turn (X)")
                if not self.ai_move():  # If AI has no moves
                    break

if __name__ == '__main__':
    game = Checkers()
    game.play()

Welcome to Simplified Checkers!
You are O (green), AI is X (red)
Enter moves like: '5,2 4,3' to move from (5,2) to (4,3)
Player pieces (O) move upward, AI pieces (X) move downward

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  -   -   -   -   -   -   -   -   
4  -   -   -   -   -   -   -   -   
5  O   -   O   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  5,0 4,1



   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  -   -   -   -   -   -   -   -   
4  -   O   -   -   -   -   -   -   
5  -   -   O   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI moves from (2,1) to (3,0)

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   -   -   X   -   X   -   X   
3  X   -   -   -   -   -   -   -   
4  -   O   -   -   -   -   -   -   
5  -   -   O   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  5,2 4,3



   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   -   -   X   -   X   -   X   
3  X   -   -   -   -   -   -   -   
4  -   O   -   O   -   -   -   -   
5  -   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI moves from (1,0) to (2,1)

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  -   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   -   -   -   -   -   -   
4  -   O   -   O   -   -   -   -   
5  -   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


...

Invalid move! Try again.


Enter your move (from_row,from_col to_row,to_col):  4,7 3,6



   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   -   -   X   -   X   -   X   
3  X   -   X   -   -   -   O   -   
4  -   O   -   O   -   O   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI moves from (1,0) to (2,1)

   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  -   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   X   -   -   -   O   -   
4  -   O   -   O   -   O   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  4,5 3,4



   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  -   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   X   -   O   -   O   -   
4  -   O   -   O   -   -   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   


   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  -   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   X   -   O   -   O   -   
4  -   O   -   O   -   -   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI has no valid moves left. You win!


In [ ]:
from copy import deepcopy
import time
import math

# Color codes for better visualization
ansi_red = "\u001b[31m"
ansi_green = "\u001b[32m"
ansi_reset = "\u001b[0m"

class Checkers:
    def __init__(self):
        self.matrix = [[], [], [], [], [], [], [], []]
        self.current_turn = True  # True for player (O), False for AI (X)
        self.player_pieces = 12
        self.ai_pieces = 12
        self.max_depth = 3  # Depth for Minimax search
        
        # Initialize empty board
        for row in self.matrix:
            for _ in range(8):
                row.append("---")
        
        # Position pieces
        self.position_pieces()

    def position_pieces(self):
        # Place AI pieces (X) at top (rows 0-2)
        for i in range(3):
            for j in range(8):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = f"X{i}{j}"  # AI pieces (X)
        
        # Place player pieces (O) at bottom (rows 5-7)
        for i in range(5, 8):
            for j in range(8):
                if (i + j) % 2 == 1:
                    self.matrix[i][j] = f"O{i}{j}"  # Player pieces (O)

    def print_board(self):
        print("\n   " + "   ".join(str(i) for i in range(8)))
        for i, row in enumerate(self.matrix):
            print(i, end="  ")
            for cell in row:
                if cell[0] == 'X':
                    print(ansi_red + cell[0] + ansi_reset, end="   ")
                elif cell[0] == 'O':
                    print(ansi_green + cell[0] + ansi_reset, end="   ")
                else:
                    print(cell[0], end="   ")
            print()
        print()

    def get_player_move(self):
        while True:
            try:
                move = input("Enter your move (from_row,from_col to_row,to_col): ").strip()
                if not move:
                    print("Game ended!")
                    exit()
                
                from_pos, to_pos = move.split()
                old_i, old_j = map(int, from_pos.split(','))
                new_i, new_j = map(int, to_pos.split(','))
                
                # Validate move
                if self.is_valid_move(old_i, old_j, new_i, new_j, is_player=True):
                    self.make_move(old_i, old_j, new_i, new_j)
                    return
                else:
                    print("Invalid move! Try again.")
            except (ValueError, IndexError):
                print("Invalid input format! Use: from_row,from_col to_row,to_col")

    def is_valid_move(self, old_i, old_j, new_i, new_j, is_player):
        # Check if coordinates are within bounds
        if not (0 <= old_i < 8 and 0 <= old_j < 8 and 0 <= new_i < 8 and 0 <= new_j < 8):
            return False
        
        # Check if moving from own piece
        piece = self.matrix[old_i][old_j][0]
        if is_player and piece != 'O':
            return False
        if not is_player and piece != 'X':
            return False
        
        # Check if destination is empty
        if self.matrix[new_i][new_j] != "---":
            return False
        
        # Check if move is diagonal
        if abs(old_i - new_i) != 1 or abs(old_j - new_j) != 1:
            return False
        
        # Check direction (player O moves up, AI X moves down)
        if is_player and new_i >= old_i:  # Player can only move up (decreasing row)
            return False
        if not is_player and new_i <= old_i:  # AI can only move down (increasing row)
            return False
            
        return True

    def make_move(self, old_i, old_j, new_i, new_j):
        # Move the piece
        piece = self.matrix[old_i][old_j]
        self.matrix[old_i][old_j] = "---"
        self.matrix[new_i][new_j] = piece
        
        # Switch turns
        self.current_turn = not self.current_turn
        
        # Update piece counts
        self.update_piece_counts()

    def update_piece_counts(self):
        self.player_pieces = sum(1 for row in self.matrix for cell in row if cell[0] == 'O')
        self.ai_pieces = sum(1 for row in self.matrix for cell in row if cell[0] == 'X')

    def evaluate_board(self):
        """
        Evaluate the board from AI's perspective.
        Higher score means better position for AI.
        """
        score = 0
        ai_pieces = 0
        player_pieces = 0
        
        for i in range(8):
            for j in range(8):
                piece = self.matrix[i][j][0]
                if piece == 'X':
                    ai_pieces += 1
                    # Encourage central control
                    if 2 <= i <= 5 and 2 <= j <= 5:
                        score += 1
                    # Encourage advancing (closer to promotion)
                    score += (7 - i) * 0.5
                elif piece == 'O':
                    player_pieces += 1
                    # Penalize opponent's central control
                    if 2 <= i <= 5 and 2 <= j <= 5:
                        score -= 1
                    # Penalize opponent's advancement
                    score -= i * 0.5
        
        # Main evaluation - piece difference with large weight
        score += (ai_pieces - player_pieces) * 10
        
        return score

    def minimax(self, board, depth, alpha, beta, maximizing_player):
        """
        Minimax algorithm with Alpha-Beta pruning
        Returns: evaluation score of the board
        """
        if depth == 0 or self.is_terminal(board):
            return self.evaluate_board()
        
        if maximizing_player:  # AI's turn (maximize score)
            max_eval = -math.inf
            for move in self.get_all_moves(board, False):  # False = AI's turn
                new_board = self.simulate_move(board, move)
                eval = self.minimax(new_board, depth-1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta cutoff
            return max_eval
        else:  # Player's turn (minimize score)
            min_eval = math.inf
            for move in self.get_all_moves(board, True):  # True = player's turn
                new_board = self.simulate_move(board, move)
                eval = self.minimax(new_board, depth-1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha cutoff
            return min_eval

    def get_all_moves(self, board, is_player):
        """Get all possible moves for the given player"""
        moves = []
        for i in range(8):
            for j in range(8):
                piece = board[i][j][0]
                if (is_player and piece == 'O') or (not is_player and piece == 'X'):
                    # Check possible moves
                    directions = [(-1, -1), (-1, 1)] if is_player else [(1, -1), (1, 1)]
                    for di, dj in directions:
                        new_i, new_j = i + di, j + dj
                        if 0 <= new_i < 8 and 0 <= new_j < 8 and board[new_i][new_j] == "---":
                            moves.append((i, j, new_i, new_j))
        return moves

    def simulate_move(self, board, move):
        """Create a new board with the move applied"""
        new_board = deepcopy(board)
        i, j, new_i, new_j = move
        piece = new_board[i][j]
        new_board[i][j] = "---"
        new_board[new_i][new_j] = piece
        return new_board

    def is_terminal(self, board):
        """Check if the game is over for the given board"""
        ai_count = sum(1 for row in board for cell in row if cell[0] == 'X')
        player_count = sum(1 for row in board for cell in row if cell[0] == 'O')
        return ai_count == 0 or player_count == 0

    def ai_move(self):
        """AI makes a move using Minimax with Alpha-Beta pruning"""
        print("AI is thinking...")
        start_time = time.time()
        
        best_move = None
        best_value = -math.inf
        alpha = -math.inf
        beta = math.inf
        
        # Get all possible moves for AI
        possible_moves = self.get_all_moves(self.matrix, False)
        
        if not possible_moves:
            print("AI has no valid moves left!")
            return False
        
        # Evaluate each possible move
        for move in possible_moves:
            new_board = self.simulate_move(self.matrix, move)
            move_value = self.minimax(new_board, self.max_depth-1, alpha, beta, False)
            
            if move_value > best_value:
                best_value = move_value
                best_move = move
                alpha = max(alpha, best_value)
        
        # Execute the best move
        if best_move:
            i, j, new_i, new_j = best_move
            print(f"AI moves from ({i},{j}) to ({new_i},{new_j})")
            print(f"Decision took {time.time()-start_time:.2f} seconds")
            self.make_move(i, j, new_i, new_j)
            return True
        
        return False

    def check_game_over(self):
        if self.player_pieces == 0:
            self.print_board()
            print("You have no pieces left. AI wins!")
            return True
        if self.ai_pieces == 0:
            self.print_board()
            print("AI has no pieces left. You win!")
            return True
        
        # Check if current player has no valid moves
        current_player = 'O' if self.current_turn else 'X'
        if not self.get_all_moves(self.matrix, self.current_turn):
            self.print_board()
            if current_player == 'O':
                print("You have no valid moves left. AI wins!")
            else:
                print("AI has no valid moves left. You win!")
            return True
                
        return False

    def play(self):
        print("Welcome to Checkers with AI!")
        print("You are O (green), AI is X (red)")
        print("Enter moves like: '5,2 4,3' to move from (5,2) to (4,3)")
        print("Player pieces (O) move upward, AI pieces (X) move downward")
        
        while True:
            self.print_board()
            
            if self.check_game_over():
                break
                
            if self.current_turn:
                print("Your turn (O)")
                self.get_player_move()
            else:
                print("AI's turn (X)")
                if not self.ai_move():  # If AI has no moves
                    break

if __name__ == '__main__':
    game = Checkers()
    game.play()

Welcome to Checkers with AI!
You are O (green), AI is X (red)
Enter moves like: '5,2 4,3' to move from (5,2) to (4,3)
Player pieces (O) move upward, AI pieces (X) move downward

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  -   -   -   -   -   -   -   -   
4  -   -   -   -   -   -   -   -   
5  O   -   O   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  5,2 4,3



   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  -   -   -   -   -   -   -   -   
4  -   -   -   O   -   -   -   -   
5  O   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI is thinking...
AI moves from (2,1) to (3,0)
Decision took 0.01 seconds

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   -   -   X   -   X   -   X   
3  X   -   -   -   -   -   -   -   
4  -   -   -   O   -   -   -   -   
5  O   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  4,3 3,2



   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   -   -   X   -   X   -   X   
3  X   -   O   -   -   -   -   -   
4  -   -   -   -   -   -   -   -   
5  O   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI is thinking...
AI moves from (1,0) to (2,1)
Decision took 0.01 seconds

   0   1   2   3   4   5   6   7
0  -   X   -   X   -   X   -   X   
1  -   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   O   -   -   -   -   -   
4  -   -   -   -   -   -   -   -   
5  O   -   -   -   O   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


...


   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   O   -   -   -   -   -   
4  -   O   -   -   -   O   -   -   
5  -   -   -   -   -   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI is thinking...
AI moves from (2,3) to (3,4)
Decision took 0.00 seconds

   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   -   -   X   -   X   
3  X   -   O   -   X   -   -   -   
4  -   O   -   -   -   O   -   -   
5  -   -   -   -   -   -   O   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  5,6 4,7



   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   -   -   X   -   X   
3  X   -   O   -   X   -   -   -   
4  -   O   -   -   -   O   -   O   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI is thinking...
AI moves from (1,2) to (2,3)
Decision took 0.00 seconds

   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   -   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   O   -   X   -   -   -   
4  -   O   -   -   -   O   -   O   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)


Enter your move (from_row,from_col to_row,to_col):  4,7 3,6



   0   1   2   3   4   5   6   7
0  -   -   -   X   -   X   -   X   
1  X   -   -   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   O   -   X   -   O   -   
4  -   O   -   -   -   O   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

AI's turn (X)
AI is thinking...
AI moves from (0,3) to (1,2)
Decision took 0.00 seconds

   0   1   2   3   4   5   6   7
0  -   -   -   -   -   X   -   X   
1  X   -   X   -   X   -   X   -   
2  -   X   -   X   -   X   -   X   
3  X   -   O   -   X   -   O   -   
4  -   O   -   -   -   O   -   -   
5  -   -   -   -   -   -   -   -   
6  -   O   -   O   -   O   -   O   
7  O   -   O   -   O   -   O   -   

Your turn (O)
